# XGBoost Model

#### What is XGBoost?

XGBoost (Extreme Gradient Boosting) is a popular gradient boosting framework that is designed to optimize performance and speed in tree ensemble learning algorithms. It uses a combination of gradient boosting and regularization techniques to prevent overfitting and improve model accuracy. XGBoost has become a popular choice for machine learning competitions and is widely used in industry for its performance and scalability.

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import get_scorer

## Functions to calculate TPR and FPR

In [2]:
def get_TPR(actual, prediction):
    TP = confusion_matrix(actual, prediction)[1][1]
    FN = confusion_matrix(actual, prediction)[1][0]
    TPR = (TP/(TP+FN))
    
    return TPR
    
def get_FPR(actual, prediction):
    FP = confusion_matrix(actual, prediction)[0][1]
    TN = confusion_matrix(actual, prediction)[0][0]
    FPR = (FP/(FP+TN))
    
    return FPR

In [3]:
data = pd.read_csv('SMData.csv')
df = data
data.columns

Index(['Unnamed: 0', ' Debt ratio %', ' Borrowing dependency',
       ' Liability to Equity', ' Net Income to Stockholder's Equity',
       ' Persistent EPS in the Last Four Seasons',
       ' ROA(C) before interest and depreciation before interest',
       ' Net worth/Assets',
       ' ROA(B) before interest and depreciation after tax',
       ' ROA(A) before interest and % after tax',
       ' Net Income to Total Assets', 'Bankrupt?'],
      dtype='object')

In [12]:
# Define the hyperparameters to tune
params = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.5],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_lambda': [0.01, 0.1, 1.0]
}

# Get a list of valid scoring options
valid_scoring = get_scorer('accuracy')
print(f"Valid scoring options: {valid_scoring}")

Valid scoring options: make_scorer(accuracy_score)


In [5]:
y = pd.DataFrame(data['Bankrupt?'])
x = df.drop(['Bankrupt?'], axis = 1)

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=2023)

In [14]:
XGBModel = XGBClassifier(objective='binary:logistic')

#Hyperparameter tuning
grid_search = GridSearchCV(estimator=XGBModel, param_grid=params, scoring=accuracy_score)
grid_search.fit(x, y)
print("Best hyperparameters: ", grid_search.best_params_)
print("Best evaluation metric: ", grid_search.best_score_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\_param_validation.py", line 175, in wrapper
    params = func_sig.bind(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\inspect.py", line 3045, in bind
    return self._bind(args, kwargs)
  File "C:\ProgramData\Anaconda3\lib\inspect.py", line 2971, in _bind
    raise TypeError(
TypeError: too many positional arguments

  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan 

Best hyperparameters:  {'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 1, 'reg_lambda': 0.01, 'subsample': 0.6}
Best evaluation metric:  nan


In [ ]:
XGBModel.fit(x_train, y_train)

y_pred = XGBModel.predict(x_test)

In [ ]:
print("XGBoost Classifier Confusion Matrix")
sns.heatmap(confusion_matrix(y_test, y_pred), 
           annot=True, fmt='.0f', annot_kws={"size":18})

print("Classification Accuracy: ", XGBModel.score(x_test, y_test))
print('TPR: ', get_TPR(y_test, y_pred))
print('FPR: ', get_FPR(y_test, y_pred))

In [ ]:
print('XGBoost TEST F1 Score: {}'.format(f1_score(y_test,y_pred)))